Iowa
----

<img src="https://github.com/computationaljournalism/columbia2020/blob/master/images/live.jpeg?raw=true" width=500>

For homework, we are going to look at Iowa a little more closely (since the IDP was unable to pull it together Monday night). The New York Times has made a kind of dashboard for the Democatic primaries that you can examine [here](https://www.nytimes.com/interactive/2020/02/04/us/elections/results-iowa-caucus.html). As you scroll around, you see computation everywhere. From predictions (the dreaded needle) to maps of "live" results. 

Last time we wanted to find data from Iowa as it came in, and then perhaps repeat this process for the other primaries, looking county-by-county. First off, the page that the New York Times uses to show its results is continually updating. I mean every 10 seconds, not that the IDP is acting that quickly. How is the data being communicated?

Let's look! (Insert an insightful few minutes with Chrome's "Developer Console", tracking the objects that the New York Times graphic is displaying.)

Now that we've found the data source, we can pull the data into our notebook using the `requests` library. `requests` lets you, through Python, make the same kinds of file access as you can in your browser. We'll have A LOT to say about pulling data from the web, but for now `requests` will let you grab a file given its URL. Here's how easy it is.

In [ ]:
from requests import get

url = 'https://int.nyt.com/applications/elections/2020/data/api/2020-02-03/iowa/president/democrat.json'
response = get(url)

type(response)

The object that `get()` returns is of type `Response`. It contains methods and data, like any Python object. The data we requested from the New York Times is a JSON file (JSON stands for JavaScript Object Notation). As we have seen, JSON is a data format that is essentially assembled from dictionaries and lists and other built-in Python types (with some slight notational differences. We can construct a Python version of the data using the method `.json()` of the `response` object. Like this!

In [ ]:
demo = response.json()
type(demo)

A dictionary! So the data coming from the New York Times to power their graphic is essentially a dictionary. How do we see what's there?

In [ ]:
demo.keys()

And let's dig in. We can look for the `.keys()` of this dictionary...

In [ ]:
demo['data'].keys()

... and see what's under `races`.

In [ ]:
type(demo["data"]["races"])

In [ ]:
len(demo["data"]["races"])

OK a list of one entry. (Only one race so far maybe?) What is that one entry?

In [ ]:
type(demo['data']['races'][0])

So all that is a dictionary. Let's look at the keys. (`sorted()` sorts the entries of a list for us.)

In [ ]:
sorted(demo['data']['races'][0].keys())

And we can sample around a little. The entries `"counties"` seems like it might be promising.

In [ ]:
type(demo['data']['races'][0]['counties'])

In [ ]:
len(demo['data']['races'][0]['counties'])

So that means we have a list of counties - 104 in total. Each entry is a...

In [ ]:
type(demo['data']['races'][0]['counties'][0])

... dictionary! So after all that, the counties are stored in a list, each element of the list being a dictionary - one dictionary per county. Let's have a look at the data stored about the first county (element 0). We are going to use the `pprint()` or pretty print function to make a slightly nicer display of the dictionary. To make this a little easier, we are going to take all our digging and store the data in a list called `"county_list"`. 

We first import the `pprint()` function from the `pprint` package. Pretty printing will add indentation to the display of the data so that you can more clearly see where dictionaries start and end, say.

So, here's one county. 

In [ ]:
from pprint import pprint

county_list = demo['data']['races'][0]['counties']

pprint(county_list[0])

According to the Times, the `results` dictionary contains "candidate totals" - these are "state delegate equivalents, which are derived from caucus vote tallies and determine the number of pledged delegates each candidate receives." So when we ran this code, for the first county, Polk, so far 391.16 delegate equivalents had been awarded...

In [ ]:
county_list[0]['votes']

... 102.97 of them to Buttigieg. 

In [ ]:
county_list[0]['results']['buttigiegp']

We also see all 177 of 177 precincts had reported - given all the changes from the IDP these results may not match what you see. 

In [ ]:
print(county_list[0]['reporting'],county_list[0]['precincts'])

**Question** If we want to print out 10 of the counties in Iowa, how would we do that? Would we do something like this?

In [ ]:
print(county_list[0]["name"], county_list[0]["reporting"], county_list[0]["precincts"])
print(county_list[1]["name"], county_list[1]["reporting"], county_list[1]["precincts"])
print(county_list[2]["name"], county_list[2]["reporting"], county_list[2]["precincts"])
print(county_list[3]["name"], county_list[3]["reporting"], county_list[3]["precincts"])
print(county_list[4]["name"], county_list[4]["reporting"], county_list[4]["precincts"])
print(county_list[5]["name"], county_list[5]["reporting"], county_list[5]["precincts"])
print(county_list[6]["name"], county_list[6]["reporting"], county_list[6]["precincts"])
print(county_list[7]["name"], county_list[7]["reporting"], county_list[7]["precincts"])
print(county_list[8]["name"], county_list[8]["reporting"], county_list[8]["precincts"])
print(county_list[9]["name"], county_list[9]["reporting"], county_list[9]["precincts"])

What if we needed to print out 100 counties instead of 10? This doesn't seem like the right approach, copying and pasting instructions.

## Introducing Loops! 

Loops are available in most programming languages and they simply allow code to be executed repeatedly. Let's see what this means by looking at an example.

In [ ]:
# Introducing Loops! 
# say we have a list with three names (strings) in it
staff = ['mark', 'eugene', 'mike']

# we can print out the length of the list
print("The list has", len(staff), "elements")

# we can also print out the 1st, 2nd and 3rd element in the list
print(staff[0])
print(staff[1])
print(staff[2])

This is tedious! 

So instead we will consider a loop. In Python a so-called "for loop" starts with a line that begins with the keyword `for` and ends in a colon. We then use **indentation** to indicate which lines are to be executed as the loop runs. (If you have programmed before, languages use different ways of representing a series of commands is to be repeated in a loop. Guido chose indentation as his coding convention because it makes the loop more readable.)

First, let's try it out.

In [ ]:
# the following is an example of a "for" loop
# we will "loop through" the list "staff" and print out each instructor's name

for teacher in staff:
    # start of the code to run each time we go through the loop
    print(teacher)
    # end of the code to run each time we go through the loop

In the loop above, we ran through the elements in the list `staff`. For each we will just call `print()`. The loop will execute three times, one for each of your instructor's names. With each pass of the loop, the variable `teacher` will be associated with a value in the `staff` list. 

So, on the first pass of the loop, the variable `teacher` refers to the first element of `staff` - the string `"mark"`. And when we `print(teacher)` we should see that name. On the second pass, the variable `teacher` refers to the second element of the list `staff` - the string `"eugene"`. When we `print(teacher)` we should see that name.

Easy!

The `print()` command is indented after the introductory line that starts with a `for` and ends with a `:`. All the lines of the loop need to be indented by the same amount (Guido doesn't care how many spaces or tabs, just that each line that you want executed in the loop is indented the same way. Three spaces, six... I just use the notebook's default.)

We use indentation to show the end of the loop also. So in the example below, we print each person's name, but the last `print()` command is only executed once - it is not part of the loop because it is not indented.

In [ ]:
for teacher in staff:
    print(teacher)

print("all done!")

If we indent the last line above and put it at the same level as  `print(teacher)`, then it's part of the loop and gets executed three times.

In [ ]:
for teacher in staff:
    print(teacher)

    print("all done!")

**Back to the primaries**

Let's put a `for` loop to use with some counties in Iowa. Remember our county list. Let's just print out the county names first.

In [ ]:
# Remember our county list...
county_list = demo['data']['races'][0]['counties']

# loop over the tweets and print out the tweet text
for county in county_list:
    print(county["name"])

Now let's print out the county name followed by some data on the county (the fraction of precincts reporting) with a dashed line to separate the different entries. (Recall that if you take a string like `"-"` and multiply it by 10, you get `"----------"`.)

In [ ]:
# loop over the counties and print out...
for county in county_list:
    
    # the county name
    print(county["name"])
    
    # the percentage of the precincts reporting
    print(county["reporting"]/county["precincts"])
    
    # and a line to indicate we're moving on to the next county
    print("-"*10)

Let's tally the votes for Buttigieg and for Sanders. We'll store them in variables called `sanders` and `buttigieg`. We begin by setting each to zero (no votes) and then loop over the counties, for each county adding its votes for the two candidates.  

In [ ]:
# create two variables that start with 0 votes for each candidate
buttigieg = 0
sanders = 0

# then loop over the counties and tally up how many votes each candidate received

for county in county_list:
    
    # for each candidate take the total for the previous counties and add their votes in the current county
    
    buttigieg = buttigieg + county["results"]["buttigiegp"]
    sanders = sanders + county["results"]["sandersb"]
    
# the loop ends - print things out

print("Buttigieg has",buttigieg,"votes")
print("Sanders has",sanders,"votes")

The code below does the same thing, but uses a slightly different notation - we use the operator `+=`. Technically, the expression `sanders += 10` will increment the variable `sanders` by 10. So the lines below increment each candidate's total like we did above. Is it more readable?

In [ ]:
# create two variables that start with 0 votes for each candidate
buttigieg = 0
sanders = 0

# then loop over the counties and tally up how many votes each candidate received

for county in county_list:
    
    # for each candidate take the previous total and add their votes in the current county
    
    buttigieg += county["results"]["buttigiegp"]
    sanders += county["results"]["sandersb"]
    
# the loop ends - print things out

print("Buttigieg has",buttigieg,"votes")
print("Sanders has",sanders,"votes")

**A. Use the example above to tally the votes for Warren and for Biden.**

In [ ]:
# Your code here!



OK so now let's print out the vote totals for everyone in each county. Remember these are given as a dictionary under the key `"results"`. 

In [ ]:
for county in county_list:
    
    print(county["results"])
    print("-"*10)

With a small change to the loop we can pull the dictionaries of `"results"`, but store them in a `list` rather than just printing them. Remember we can add data to a list with the `append()` method. Here we go!

In [ ]:
# create an empty list
vote_totals = []

for county in county_list:
    
    # for each pass of the loop add an entry to the list representing a county's vote totals
    vote_totals.append(county["results"])

# how many dictionaries of vote results are in the list?
len(vote_totals)

In [ ]:
# show the votes in the fourth county
vote_totals[3]

So, now we have a list of dictionaries, one dictionary per county. Hmm. A list of dictionaries. We can just toss that baby right into a call to `DataFrame()` to make a proper table!

In [ ]:
from pandas import DataFrame
iowa = DataFrame(vote_totals)

iowa

In [ ]:
print("HA! "*100)

Good, right?!?! OK so in the last loop (a couple cells above), we created a list called `vote_totals` by running through the `county_list` and pulling out the `"results"` dictionary from each entry and appending it to the list. 

**B. Mimic this, but create an empty list called `county_names` that we will use to store, well, each county name. Then make a loop that appends the name of the counties as it goes. This should leave you with a list of 104 county names.**

In [ ]:
# Your code here



**C. Finally, we can add the `county_names` data you created to the `iowa` DataFrame. Remember how we add columns?**

In [ ]:
# your code here




Finally, have a look. We can, by the way, sort the table by columns. Here we use `.sort_values()` to sort according to the `buttigiegp` counts (with `ascending=False` to mean that we are starting from biggest down to smallest rather than the other way.

In [ ]:
iowa.sort_values("buttigiegp",ascending=False)

**D. Your data! What data did you find? Try bringing it into the notebook!**

In [ ]:
# Your code here


